In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [3]:
# llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm=ChatOpenAI(model="gpt-4o-mini")

In [4]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        SystemMessagePromptTemplate.from_template(
            template="""
                Write out the following equation using algebric symbols then solve it. Use the format
                EQUATION:...
                SOLUTION:...
            """
        ),
        HumanMessagePromptTemplate.from_template(
            template="{equation_statement}"
        )
    ]
)

In [5]:
statement="5 times x plus 3 times y equals 7; Negative 3 times x plus 5 times y equals 23"

In [6]:

chain={"equation_statement":RunnablePassthrough()}|prompt|llm|StrOutputParser()

In [7]:
print(chain.invoke(input=statement))

EQUATION: 
1) \( 5x + 3y = 7 \)  
2) \( -3x + 5y = 23 \)  

To solve the system of equations, we can use the method of substitution or elimination. Here, we'll use the elimination method.

First, we can multiply the first equation by \( 3 \) and the second equation by \( 5 \) to align the coefficients of \( y \):

1) \( 15x + 9y = 21 \)  (by multiplying the first equation by \( 3 \))  
2) \( -15x + 25y = 115 \) (by multiplying the second equation by \( 5 \))

Now, we can add the two equations together:

\[
(15x + 9y) + (-15x + 25y) = 21 + 115
\]

This simplifies to:

\[
34y = 136
\]

Now, divide both sides by \( 34 \):

\[
y = 4
\]

Now that we have \( y \), we can substitute it back into one of the original equations to solve for \( x \). Using the first equation:

\[
5x + 3(4) = 7
\]
\[
5x + 12 = 7
\]
\[
5x = 7 - 12
\]
\[
5x = -5
\]
\[
x = -1
\]

Therefore, the solution to the system of equations is:

SOLUTION: \( x = -1 \), \( y = 4 \)


In [ ]:
k=RunnablePassthrough.\
    assign(equation_statement=itemgetter('input_phrase')|prompt).\
    assign(response=itemgetter('equation_statement')|llm).\
    assign(response=itemgetter('response')|StrOutputParser())|RunnableLambda(lambda x:x['response'])

k.invoke(input={'input_phrase':statement})

"EQUATION:  \n1) \\( 5x + 3y = 7 \\)  \n2) \\( -3x + 5y = 23 \\)  \n\nTo solve this system of equations, we can use the substitution or elimination method. Here, we'll use the elimination method.\n\nFirst, we can multiply the first equation by 3 and the second equation by 5 to align the coefficients of \\(y\\):\n\nMultiply equation 1 by 3:  \n\\( 15x + 9y = 21 \\)  (Equation 3)\n\nMultiply equation 2 by 5:  \n\\( -15x + 25y = 115 \\)  (Equation 4)\n\nNow we add Equations 3 and 4 to eliminate \\(x\\):\n\n\\( 15x + 9y + (-15x + 25y) = 21 + 115 \\)  \nThis simplifies to:  \n\\( 34y = 136 \\)  \n\nNow divide by 34:  \n\\( y = 4 \\)\n\nNow substitute \\(y = 4\\) back into one of the original equations, we will use Equation 1:\n\n\\( 5x + 3(4) = 7 \\)  \n\\( 5x + 12 = 7 \\)  \nNow subtract 12 from both sides:  \n\\( 5x = -5 \\)  \n\nDivide by 5:  \n\\( x = -1 \\)\n\nNow we have the values of \\(x\\) and \\(y\\):\n\nSOLUTION:  \n\\( x = -1 \\), \\( y = 4 \\)"

<h3> Using the Stop Keywords</h3>

In [16]:
chain={"equation_statement":RunnablePassthrough()}|prompt|llm.bind(stop="SOLUTION")|StrOutputParser()

In [17]:
print(chain.invoke(input=statement))

EQUATION:  
1) \( 5x + 3y = 7 \)  
2) \( -3x + 5y = 23 \)

To solve this system of equations, we can use the substitution or elimination method. Here, I will use the elimination method.

First, let's multiply the first equation by 3 and the second equation by 5 to align the coefficients of \(x\):

1) \( 3(5x + 3y) = 3(7) \)  
   \( 15x + 9y = 21 \)   (Equation 3)

2) \( 5(-3x + 5y) = 5(23) \)  
   \( -15x + 25y = 115 \)   (Equation 4)

Now we will add Equation 3 and Equation 4:

\[
(15x + 9y) + (-15x + 25y) = 21 + 115
\]
\[
34y = 136
\]
\[
y = \frac{136}{34} = 4
\]

Now substitute \( y = 4 \) back into the first original equation to find \( x \):

\[
5x + 3(4) = 7
\]
\[
5x + 12 = 7
\]
\[
5x = 7 - 12
\]
\[
5x = -5
\]
\[
x = \frac{-5}{5} = -1
\]

Thus, the solution to the system of equations is:




In [12]:
tools=[
        {
            "type":"function",
            "function": 
                {
                    "name": "solver",
                    "description":"Formulates and Solves an Equation",
                    "parameters":
                        {
                            "type":"object",
                            "properties":
                                {
                                    "equation":
                                        {
                                            "type":"string",
                                            "description":"The Algebric Expression of the Equation"
                                        },
                                    "solution":
                                        {
                                            "type":"string",
                                            "description":"The Solution to the Equation"
                                        }
                                },
                            "required":["equation","solution"]
                        },
                },
        }
    ]

In [13]:
chain={"equation_statement":RunnablePassthrough()}|prompt|llm.bind(tools=tools)

In [14]:
chain.invoke(input=statement).pretty_print()

================================== Ai Message ==================================
Tool Calls:
  solver (call_8HKt47JTVufa9ynBzCvKlwoT)
 Call ID: call_8HKt47JTVufa9ynBzCvKlwoT
  Args:
    equation: 5x + 3y = 7
    solution: x = 2, y = -1
  solver (call_yF0KmKMcU3KdOJM2VvJZGSKB)
 Call ID: call_yF0KmKMcU3KdOJM2VvJZGSKB
  Args:
    equation: -3x + 5y = 23
    solution: x = 2, y = -1
